<a href="https://colab.research.google.com/github/sofiamln/03MIAR---Algoritmos-de-Optimizacion/blob/main/AG3/Sof%C3%ADa_Molina_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Actividad guiada 2 de Algoritmos de Optimización


Sofía Molina Arias 
Link: https://colab.research.google.com/drive/1F6hSQuB21f-H1Kt5zRhQjF_L-2s1f_8b?usp=sharing

Github: https://github.com/sofiamln/03MIAR---Algoritmos-de-Optimizacion.git

In [1]:
!pip install requests    #Hacer llamadas http a paginas de la red
!pip install tsplib95    #Modulo para las instancias del problema del TSP

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.9 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.0
    Uninstalling networkx-3.0:
      Successfully uninstalled networkx-3.0


In [2]:
#Cargamos los datos del problema

import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95       #Modulo para las instancias del problema del TSP
import math           #Modulo de funciones matematicas. Se usa para exp

In [4]:
#Ahora descargamos el ficheroy

file = "swiss42.tsp" ; 
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz     #Descomprimir el fichero de datos 


gzip: swiss42.tsp already exists; do you wish to overwrite (y or n)? y


In [5]:
#Importamos la librería random

import random 

In [7]:
#Cargamos los datos

problem = tsplib95.load(file)

#e imprimimos el problema
problem

In [10]:
#Queremos ver la distandia de ir del nodo 0 al nodo 1 por ejemplo

print(problem.get_weight(0,1))

#O del 0 al 3

print(problem.get_weight(0,3))

23

In [14]:
#también podemos imprimir la lista de los nodos
Nodos = list(problem.get_nodes())
print(Nodos)

#O las aristas
Aristas = list(problem.get_edges())
#print(Aristas)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]


In [ ]:
def crear_solucion(Nodos):
  solucion=[Nodos[0]]
  for n in Nodos[1:]:
    solucion+=[random.choice(list(set(Nodos)-set({Nodos[0]}-set(solucion))))]

  return solucion

crear_solucion(Nodos)

In [23]:
#Creamos un función que calcule la distancia entre dos nodos, para simplificar la notación

def distancia(a,b,problem):
  return problem.get_weight(a,b)

In [19]:
#Creamos una función para calcular la distancia total

def distancia_total(solucion,problem):
  distancia_total=0
  for i in range(len(solucion)-1):
    distancia_total+=distancia(solucion[i],solucion[i+1],problem)
  return distancia_total+distancia(solucion[len(solucion)-1],solucion[0],problem)

In [27]:
s_random=crear_solucion(Nodos)

print(distancia_total(s_random,problem))

4757


In [59]:
#Creamos una funcion que genere N soluciones aleatorias, para quedarnos con la mejor

def busqueda_aleatoria(problem, N):
  #N es el número de iteraciones

  Nodos = list(problem.get_nodes())
  
  mejor_solucion = []                       #Inicializamos con un valor alto
  mejor_distancia = float('inf')                    #Inicializamos con un valor alto
  
  for i in range(N):                                #Criterio de parada: repetir N veces pero podemos incluir otros
    solucion = crear_solucion(Nodos)                #Genera una solucion aleatoria
    distancia = distancia_total(solucion, problem)  #Calcula el valor objetivo(distancia total)
    
    if distancia < mejor_distancia:                 #Compara con la mejor obtenida hasta ahora
      mejor_solucion = solucion
      mejor_distancia = distancia
      
      
  print("Mejor solución:" , mejor_solucion) 
  print("Distancia     :" , mejor_distancia) 
  return mejor_solucion  

    
#Busqueda aleatoria con 5000 iteraciones    
solucion = busqueda_aleatoria(problem, 5000) 

Mejor solución: [0, 37, 1, 27, 7, 4, 16, 15, 33, 36, 11, 11, 12, 30, 39, 13, 10, 30, 29, 1, 6, 26, 14, 37, 3, 20, 30, 2, 7, 7, 3, 6, 22, 8, 27, 6, 15, 1, 4, 16, 19, 17]
Distancia     : 2966


## Búsqueda Local 2-opt (2 nodos)

In [60]:
def generar_vecina(solucion):

  mejor_solucion=[]
  mejor_distancia=10e100

  for i in range(1,len(solucion)-1):
    for j in range(i+1,len(solucion)-1):

      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

      distancia_vecina = distancia_total(vecina,problem)

      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina

  return mejor_solucion

In [86]:
def busqueda_local(problem):

  mejor_solucion=[]

  solucion_referencia = crear_solucion(Nodos)
  mejor_distancia = distancia_total(solucion_referencia, problem)

  iteracion=0             #Un contador para saber las iteraciones que hacemos
  while(1):
    iteracion +=1         #Incrementamos el contador

    #Obtenemos la mejor vecina ...
    vecina = generar_vecina(solucion_referencia)

    #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
    distancia_vecina = distancia_total(vecina, problem)
 
    #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina                   #Guarda la mejor solución encontrada
      mejor_distancia = distancia_vecina

    else:
      print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejor_solucion) 
      print("Distancia     :" , mejor_distancia) 
      return mejor_solucion
      
    solucion_referencia = vecina
 
 
sol = busqueda_local(problem)  

En la iteracion  27 , la mejor solución encontrada es: [0, 18, 18, 18, 4, 4, 3, 3, 2, 32, 32, 32, 34, 31, 17, 17, 7, 7, 15, 16, 14, 14, 26, 12, 11, 10, 2, 28, 8, 9, 21, 24, 22, 34, 33, 33, 35, 35, 36, 17, 1, 27]
Distancia     : 1422
